In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Assignment 2

In [2]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from collections import OrderedDict as odict
from graphviz import Digraph
from tabulate import tabulate

### Data

In [59]:
df = pd.read_csv('data.csv')
df

,Unnamed: 0,reliable_sensor1,reliable_sensor2,reliable_sensor3,reliable_sensor4,unreliable_sensor1,unreliable_sensor2,unreliable_sensor3,unreliable_sensor4,robot1,...,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,0,no motion,no motion,no motion,no motion,no motion,motion,no motion,no motion,"('r1', 0)",...,0,0,0,0,0,0,0,0,0,17
1,1,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)",...,0,0,0,0,1,0,0,0,0,15
2,2,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)",...,0,0,0,0,1,0,0,0,0,0
3,3,no motion,no motion,motion,no motion,no motion,motion,no motion,no motion,"('r1', 0)",...,0,0,0,0,5,0,0,0,0,0
4,4,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)",...,0,0,0,0,11,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,2396,no motion,no motion,no motion,no motion,motion,motion,no motion,motion,"('c4', 0)",...,0,0,0,0,0,0,0,0,0,23
2397,2397,no motion,no motion,no motion,no motion,no motion,no motion,no motion,no motion,"('c4', 0)",...,0,0,0,0,0,0,0,0,0,23
2398,2398,no motion,no motion,no motion,motion,motion,no motion,no motion,no motion,"('r35', 0)",...,0,0,0,0,0,0,0,0,0,23
2399,2399,no motion,no motion,no motion,no motion,no motion,no motion,no motion,no motion,"('c4', 0)",...,0,0,0,0,0,0,0,0,0,23


In [60]:
for col in df.columns:
    L = list(df[col].unique())
    if None in L:
        print(col)

We need to perform the following mappings:

  -  If a sensor detects `motion`, then we map to `1`; if it detects `no motion` then map to `0`.
  -  If a room is occupied (val > 0), then we map to `1`; else, unchanged. 

In [61]:
for col in df.columns:
    if col == 'Unnamed: 0':
        continue
    if col[:5] == 'relia' or col[:5] == 'unrel':
        df[col] = df[col].map(lambda x : 1 if x == 'motion' else 0)
    if col[:5] == 'door_':
        df[col] = df[col].map(lambda x : 0 if x == 0 else 1)
    if col[:5] == 'robot':
        df[col] = df[col].map(lambda x : eval(x))
        df[col] = df[col].map(lambda x: (x[0], 1) if x[1] > 0 else (x[0], 0))
    if col[1].isnumeric() or col=='outside':
        df[col] = df[col].map(lambda x : 1 if x > 0 else 0)
df

,Unnamed: 0,reliable_sensor1,reliable_sensor2,reliable_sensor3,reliable_sensor4,unreliable_sensor1,unreliable_sensor2,unreliable_sensor3,unreliable_sensor4,robot1,...,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,0,0,0,0,0,0,1,0,0,"(r1, 0)",...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,0,1,0,0,0,0,1
2,2,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,0,1,0,0,0,0,0
3,3,0,0,1,0,0,1,0,0,"(r1, 0)",...,0,0,0,0,1,0,0,0,0,0
4,4,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,2396,0,0,0,0,1,1,0,1,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1
2397,2397,0,0,0,0,0,0,0,0,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1
2398,2398,0,0,0,1,1,0,0,0,"(r35, 0)",...,0,0,0,0,0,0,0,0,0,1
2399,2399,0,0,0,0,0,0,0,0,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1


In [62]:
#corridor 3 has been split in two: we will turn col c3 into two identical c3a and c3b. 
new_columns = {col : col for col in df.columns}
new_columns['c3'] = 'c3a'
df = df.rename(columns=new_columns)
df['c3b'] = df['c3a']
column_order = list(df.columns)
index = column_order.index('c3a')
column_order.remove('c3b')
column_order.insert(index + 1, 'c3b')
df = df[column_order]
df

,Unnamed: 0,reliable_sensor1,reliable_sensor2,reliable_sensor3,reliable_sensor4,unreliable_sensor1,unreliable_sensor2,unreliable_sensor3,unreliable_sensor4,robot1,...,r33,r34,r35,c1,c2,c3a,c3b,c4,o1,outside
0,0,0,0,0,0,0,1,0,0,"(r1, 0)",...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,1,0,0,0,0,0,1
2,2,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,1,0,0,0,0,0,0
3,3,0,0,1,0,0,1,0,0,"(r1, 0)",...,0,0,0,1,0,0,0,0,0,0
4,4,0,0,1,0,0,0,0,0,"(r1, 0)",...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,2396,0,0,0,0,1,1,0,1,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1
2397,2397,0,0,0,0,0,0,0,0,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1
2398,2398,0,0,0,1,1,0,0,0,"(r35, 0)",...,0,0,0,0,0,0,0,0,0,1
2399,2399,0,0,0,0,0,0,0,0,"(c4, 0)",...,0,0,0,0,0,0,0,0,0,1


In [58]:
from random import choice

def func(x):
    if x[0] == 'c3':
        return (choice(['c3a', 'c3b']), x[1])
    else:
        return x
    
df['robot1'] = df['robot1'].map(lambda x : func(x))
df['robot2'] = df['robot2'].map(lambda x : func(x))
df.iloc[3,:]

Unnamed: 0                   3
reliable_sensor1             0
reliable_sensor2             0
reliable_sensor3             1
reliable_sensor4             0
unreliable_sensor1           0
unreliable_sensor2           1
unreliable_sensor3           0
unreliable_sensor4           0
robot1                 (r1, 0)
robot2                (c3b, 0)
door_sensor1                 0
door_sensor2                 0
door_sensor3                 0
door_sensor4                 0
time                  08:01:00
electricity_price       0.9604
r1                           0
r2                           0
r3                           0
r4                           0
r5                           0
r6                           0
r7                           0
r8                           0
r9                           0
r10                          0
r11                          0
r12                          0
r13                          0
r14                          0
r15                          0
r16     

### Helper Functions

In [63]:
def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))
    
def factor_to_csv(f):
    """
    argument
    `f`, a factor to save to a csv file.
    The first row of the csv file will be the variables; followed by the factor
    """
    cols = list(f['dom'])
    cols.append('')
    O = f['table']
    A = np.zeros((len(O), len(cols)))
    for i, (key, val) in enumerate(O.items()):
        A[i] = [*key, val]
    df = pd.DataFrame(A, columns=cols)
    df.to_csv('_'.join(cols)+'.csv', index=False, header=True)

### Learning Model Parameters

We need to learn the following:

   -  $B(R_i^0)=P(R_i^0)$
   -  $P(R_i^{t+1} | {\rm Adj}(R_i,t))$
   -  $P(e_t|R_i^t)$
   -  $P(W_i)$ where $W$ stands for robot $i$.

First, we define the adjacency ${\rm Adj}(R_i,t)$ of each room (we will ignore timesteps for now). 

In [64]:
Adj = {
    'r1':['r1','r2','r3'],
    'r2':['r1','r2','r4'],
    'r3':['r1','r3','r7'],
    'r4':['r2','r4','r8'],
    'r5':['r5','r6','r9','c3a'],
    'r6':['r5','r6','c3a'],
    'r7':['r3','r7','c1'],
    'r8':['r4','r8','r9'],
    'r9':['r5','r8','r9','r13'],
    'r10':['r10','c3a'],
    'r11':['r11','c3a'],
    'r12':['r12','r22','outside'],
    'r13':['r9','r13','r24'],
    'r14':['r14','r24'],
    'r15':['r15','c3a'],
    'r16':['r16','c3a'],
    'r17':['r17','c3b'],
    'r18':['r18','c3b'],
    'r19':['r19','c3b'],
    'r20':['r20','c3b'],
    'r21':['r21','c3b'],
    'r22':['r12','r22','r25'],
    'r23':['r23','r24'],
    'r24':['r13','r14','r23','r24'],
    'r25':['r22','r25','r26','c1'],
    'r26':['r25','r26','r27'],
    'r27':['r26','r27','r32'],
    'r28':['r28','c4'],
    'r29':['r29','r30','c4'],
    'r30':['r29','r30'],
    'r31':['r31','r32'],
    'r32':['r27','r31','r32','r33'],
    'r33':['r32','r33'],
    'r34':['r34','c2'],
    'r35':['r35','c4'],
    'c1':['r7','r25','c1','c2'],
    'c2':['r34','c1','c2','c4'],
    #'c3':['r5','r6','r10','r11','r15','r16','r17','r18','r19','r20','r21','c3','o1'],
    'c3a': ['r5', 'r6', 'r10', 'r11', 'r15', 'r16', 'c3a', 'c3b'],
    'c3b': ['r17', 'r18', 'r19', 'r20', 'r21', 'c3a', 'c3b', 'o1'],
    'c4':['r28','r29','r35','c2','c4','o1'],
    'o1':['c3b','c4','o1'],
    'outside':['r12','outside']
}

Areas = list(Adj.keys())

for room in Adj:
    for r in Adj[room]:
        assert(room in Adj[r])

We also need to build our Signal graph, which has an outgoing edge from a room variable to a sensor variable if the sensor can detect motion in or through that room.

In [75]:
Sig = {
    'reliable_sensor1': ['r16', 'r17', 'c3b'], 
    'reliable_sensor2': ['r5', 'r6', 'r9', 'c3a'],
    'reliable_sensor3': ['r25', 'r26', 'c1'],
    'reliable_sensor4': ['r31', 'r32'],
    'unreliable_sensor1': ['c4', 'o1'],
    'unreliable_sensor2': ['r5', 'r10', 'r15', 'c3a', 'c3b'],
    'unreliable_sensor3': ['r1', 'r2'], #r4???
    'unreliable_sensor4': ['r23', 'r24'],
    'door_sensor1': ['r8', 'r9'],
    'door_sensor2': ['c1', 'c2'],
    'door_sensor3': ['r26', 'r27'],
    'door_sensor4': ['r35', 'c4']
}

Sensors = Sig.keys()

Outcome space is a constant {0,1} for all variables (both rooms and sensors).

In [66]:
outcomeSpace = [0,1]

### Learning the factors $P(R_i^0)$, for each $i$.

In [70]:
"""
A = np.sum(df.iloc[0][Areas] > 0) / len(Areas)
for R in Areas:
    #create a factor table and save to csv
    f = {
        'dom': (R,),
        'table': odict([
            ((1,), A),
            ((0,), 1-A)
        ])
    }
    printFactor(f)
    factor_to_csv(f)
"""
c = np.sum(df.iloc[0][Areas] > 0) 
N = len(Areas)
for R in Areas:
    #create a factor table and save to csv
    f = {
        'dom': (R,),
        'table': odict([
            ((1,), (c + 1) / (N + 2)),
            ((0,), 1 - (c + 1) / (N + 2))
        ])
    }
    #printFactor(f)
    factor_to_csv(f)

"\nA = np.sum(df.iloc[0][Areas] > 0) / len(Areas)\nfor R in Areas:\n    #create a factor table and save to csv\n    f = {\n        'dom': (R,),\n        'table': odict([\n            ((1,), A),\n            ((0,), 1-A)\n        ])\n    }\n    printFactor(f)\n    factor_to_csv(f)\n"

### Learning $P(R_i^{t+1} | {\rm Adj}(R_i,t))$, for each $i$. 

This is independent of $t$.

In [71]:
def learn_next_proba_from_data(f, data):
    """
    Learns probabilities in a factor from the data. Update is done in-place.
    Note that we are learning
            P(X_t | Adj(X,t-1)) = P(X_t, Adj(X,t-1)) / P(Adj(X,t-1))
    """
    # variables
    all_vars = f['dom']
    prev_vars = f['dom'][:-1]
    R = f['dom'][-1]
    # create a dataframe with columns [X_t, Adj(X_t-1)]
    columns = f['dom']
    df = data.copy()
    df.columns = columns[:-1]
    df[R] = df[R.replace('+1', '')]
    df[R] = df[R].shift(-1)
    df = df.iloc[:-1,:]
    total = df.shape[0]
    
    all_entries = list(product(*[outcomeSpace for node in f['dom']]))
    prev_entries = list(product(*[outcomeSpace for node in f['dom'][:-1]]))
                            
    #for each (pair of) rows, increment count as appropriate
    for i in range(df.shape[0]):
        row = df.iloc[i,:]
        f['table'][tuple(row[all_vars])] += 1
    
    #smooth
    for entries in all_entries:
        c = f['table'][entries]
        f['table'][entries] = (c + 1)/(total + len(f['table']))

    #final probability
    for entries in prev_entries:
        entries0, entries1 = tuple(list(entries) + [0]), tuple(list(entries) + [1])
        N = f['table'][entries0] + f['table'][entries1]
        f['table'][entries0] /= N
        f['table'][entries1] /= N

In [72]:
for R in Areas:
    f = {}
    f['dom'] = [v + '^t' for v in Adj[R]]
    f['dom'].append(R + '^t+1')
    f['table'] = odict([])
    f_ev = {}
    for entries in product(*[outcomeSpace for node in f['dom']]):
        f['table'][entries] = 0
    learn_next_proba_from_data(f, df[Adj[R]])
    factor_to_csv(f)

### Learning $P(e_t|R_i^t)$. 

Since

$$ P(e_t|R_i^t) = \left( \prod_{e : R_i^t\in{\rm Sig}(e)}{P(e|R_i^t)} \right) \left( \prod_{e : R_i^t\not\in{\rm Sig}(e)}{P(e)} \right) $$

We need to learn, for each sensor $E$, $P(E|R_i^t)$ where $R_i^t\in{\rm Sig}(E)$ (the prior probabilities for evidence will not affect any calculation, as they are constant).
  
#### Learning $P(e)$

In [73]:
"""
for E in Sensors:
    print(E)
    A = (np.sum(df[E] == 0) + 1) / (df.shape[0] + 2)
    f = {}
    f['dom'] = (E,)
    f['table'] = odict([
        ((0,), A),
        ((1,), 1-A)
    ])
    printFactor(f)
    print()
    factor_to_csv(f)
"""

"\nfor E in Sensors:\n    print(E)\n    A = (np.sum(df[E] == 0) + 1) / (df.shape[0] + 2)\n    f = {}\n    f['dom'] = (E,)\n    f['table'] = odict([\n        ((0,), A),\n        ((1,), 1-A)\n    ])\n    printFactor(f)\n    print()\n    factor_to_csv(f)\n"

#### Learning $P(E|R_i^t)$ where $R_i^t\in{\rm Sig}(E)$

Since both variables are at the same time step, and this is assumed to be constant for all $t$, we will omit the time variable from the columns.

In [74]:
def learn_cond_ev_from_data(f, data):
    """
    Learn the conditional P(E|R), where R is in in Sig[E]
    """
    # variables
    all_vars = f['dom']
    R = f['dom'][-1]
    total = data.shape[0]
    
    all_entries = list(product(*[outcomeSpace for node in f['dom']]))
                            
    #for each (pair of) rows, increment count as appropriate
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        f['table'][tuple(row)] += 1
    
    #smooth
    for entries in all_entries:
        c = f['table'][entries]
        f['table'][entries] = (c + 1)/(total + len(f['table']))

    #final probability
    for entries in product(*[outcomeSpace]):
        entries0, entries1 = tuple(list(entries) + [0]), tuple(list(entries) + [1])
        N = f['table'][entries0] + f['table'][entries1]
        f['table'][entries0] /= N
        f['table'][entries1] /= N
        
    printFactor(f)
    print()

In [76]:
for E in Sig.keys():
    for R in Sig[E]:
        f = {}
        f['dom'] = (R, E)
        f['table'] = odict([])
        for entries in product(*[outcomeSpace for node in f['dom']]):
            f['table'][entries] = 0
        learn_cond_ev_from_data(f, df[[R,E]])
        factor_to_csv(f)

|   r16 |   reliable_sensor1 |        Pr |
|-------+--------------------+-----------|
|     0 |                  0 | 0.972772  |
|     0 |                  1 | 0.0272277 |
|     1 |                  0 | 0.0167645 |
|     1 |                  1 | 0.983236  |

|   r17 |   reliable_sensor1 |       Pr |
|-------+--------------------+----------|
|     0 |                  0 | 0.535967 |
|     0 |                  1 | 0.464033 |
|     1 |                  0 | 0.409027 |
|     1 |                  1 | 0.590973 |

|   c3b |   reliable_sensor1 |       Pr |
|-------+--------------------+----------|
|     0 |                  0 | 0.485423 |
|     0 |                  1 | 0.514577 |
|     1 |                  0 | 0.515973 |
|     1 |                  1 | 0.484027 |

|   r5 |   reliable_sensor2 |        Pr |
|------+--------------------+-----------|
|    0 |                  0 | 0.962838  |
|    0 |                  1 | 0.0371622 |
|    1 |                  0 | 0.045045  |
|    1 |                 

### Learn $P(W_i)$

In [77]:
def learn_robot_data(f, W, data):
    total = data.shape[0]
    f['dom'] = (W,)
    f['table'] = odict([
        ((0,),0),
        ((1,),0)
    ])
    for i in range(total):
        row = data.iloc[i,:]
        R = row[W][0]
        if R != 'c3':
            if row[R] == row[W][1]:
                f['table'][(1,)] += 1
            else:
                f['table'][(0,)] += 1
        else:
            if row['c3a'] == row[W][1]:
                f['table'][(1,)] += 1
            else:
                f['table'][(0,)] += 1
    f['table'][(0,)] = (f['table'][(0,)] + 1) / (total + 2)
    f['table'][(1,)] = (f['table'][(1,)] + 1) / (total + 2)
    
    printFactor(f)
    print()

In [78]:
for W in ['robot1', 'robot2']:
    f = {}
    learn_robot_data(f, W, df)
    factor_to_csv(f)

|   robot1 |          Pr |
|----------+-------------|
|        0 | 0.000416146 |
|        1 | 0.999584    |

|   robot2 |          Pr |
|----------+-------------|
|        0 | 0.000416146 |
|        1 | 0.999584    |

